In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 100)
pd.set_option('max_columns', 100)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

In [2]:
# SZE order speed compared with market
startTm = datetime.datetime.now()

statsData1 = {}
statsData2 = {}
statsData3 = {}
for col in ['account', 'date', "orderNtl", "tradeNtl", "gp", 'secid', 'vai', 'side', 'our ntl', 'market orders', 'market ntl', 'front orders', 'front ntl', 'statusLs', "TradePriceLs", "TradeQtyLs"]:
    statsData1[col] = []
    statsData2[col] = []
    statsData3[col] = []

startDate = '20201228'
endDate = '20210108'

readPath = r'L:\orderLog\result\marketPos'
dataPathLs = np.array(glob.glob(os.path.join(readPath, 'marketPos1_***.pkl')))
dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
checkData = []
for path in dataPathLs:
    thisDate = os.path.basename(path).split('.')[0].split('_')[1]
    data = pd.read_pickle(path)
    data['date'] = thisDate
    checkData += [data]
checkData = pd.concat(checkData).reset_index(drop=True)   

checkData["firstclock"] = checkData.groupby(["date", "group"])["clockAtArrival"].transform("first")
checkData = checkData[checkData["firstclock"] != checkData["clockAtArrival"]]

checkData['brokerLs'] = np.where(checkData['brokerLs'].isnull(), ' ', checkData['brokerLs'])
checkData['brokerLs'] = checkData['brokerLs'].apply(lambda x: x.split(','))
checkData['firstBroker'] = checkData['brokerLs'].apply(lambda x: x[0])
checkData['sta'] = np.where(checkData['mrstaat'] == 1000, '1. staone', np.where(
checkData['mrstaat'] == 3000, '2. statwo', np.where(
checkData['mrstaat'].isin([11000, 13000]), '3. sta300', 'else')))

def f(x):
    n = len(x[0])
    m = 0
    for i in range(n):
        m = m + x[0][i] * x[1][i] / 10000
    return m
    
checkData["tradeNtl"] = checkData[["TradePriceLs", "TradeQtyLs"]].apply(f, axis=1)

for date, dateData in checkData.groupby(['date']):
    print(date)
    for group, groupData in dateData.groupby(['group']):
        groupData = groupData.reset_index(drop=True)
        ### ignore cases where brokerNum >= 2 (around 2.5%, 7% data)
        if groupData[groupData['brokerNum'] >= 2].shape[0] == 0:
            groupData['index'] = groupData.index.values
            groupData['lastIx'] = groupData.groupby(['firstBroker'])['index'].transform('last')
            groupData['isOurs'] = np.where((groupData['index'] == groupData['lastIx']) & (groupData['brokerNum'] == 1), 1, 0)
            brokerLs = groupData[groupData['brokerNum'] == 1]['firstBroker'].unique()
            for broker in brokerLs:
                if groupData[(groupData["firstBroker"] == broker) & (groupData["isOurs"] == 1)]['sta'].values == ['2. statwo']:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData1['account'].append(broker)
                    statsData1['date'].append(date)
                    statsData1['gp'].append(group)
                    statsData1['secid'].append(groupData['secid'].values[0])
                    statsData1['vai'].append(groupData['vai'].values[0])
                    statsData1['side'].append(groupData['Side'].values[0])
                    statsData1['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData1['market orders'].append(totalOrders)
                    statsData1['market ntl'].append(totalSize)
                    statsData1['front orders'].append(frontData.shape[0])
                    statsData1['front ntl'].append(frontSize)
                    statsData1['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData1['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData1['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData1['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData1['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
                
                elif groupData[(groupData["firstBroker"] == broker) & (groupData["isOurs"] == 1)]['sta'].values == ['1. staone']:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData2['account'].append(broker)
                    statsData2['date'].append(date)
                    statsData2['gp'].append(group)
                    statsData2['secid'].append(groupData['secid'].values[0])
                    statsData2['vai'].append(groupData['vai'].values[0])
                    statsData2['side'].append(groupData['Side'].values[0])
                    statsData2['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData2['market orders'].append(totalOrders)
                    statsData2['market ntl'].append(totalSize)
                    statsData2['front orders'].append(frontData.shape[0])
                    statsData2['front ntl'].append(frontSize)
                    statsData2['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData2['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData2['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData2['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData2['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
                
                elif groupData[(groupData["firstBroker"] == broker) & (groupData["isOurs"] == 1)]['sta'].values == ['3. sta300']:
                    totalOrders = groupData[groupData['isOurs'] == 0].shape[0]
                    totalSize = groupData[groupData['isOurs'] == 0]['OrderNtl'].sum()
                    ix = groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)].index.values[0]
                    frontData = groupData.iloc[:ix]
                    frontData = frontData[frontData['isOurs'] == 0]
                    frontSize = frontData['OrderNtl'].sum()
                    statsData3['account'].append(broker)
                    statsData3['date'].append(date)
                    statsData3['gp'].append(group)
                    statsData3['secid'].append(groupData['secid'].values[0])
                    statsData3['vai'].append(groupData['vai'].values[0])
                    statsData3['side'].append(groupData['Side'].values[0])
                    statsData3['our ntl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData3['market orders'].append(totalOrders)
                    statsData3['market ntl'].append(totalSize)
                    statsData3['front orders'].append(frontData.shape[0])
                    statsData3['front ntl'].append(frontSize)
                    statsData3['statusLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['statusLs'].values[0])
                    statsData3['TradePriceLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradePriceLs'].values[0])
                    statsData3['TradeQtyLs'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['TradeQtyLs'].values[0])
                    statsData3['orderNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['OrderNtl'].values[0])
                    statsData3['tradeNtl'].append(groupData[(groupData['firstBroker'] == broker) & (groupData['isOurs'] == 1)]['tradeNtl'].values[0])
statsData1 = pd.DataFrame(statsData1)
statsData1['exchange'] = np.where(statsData1['secid'] >= 2000000, 'SZE', 'SSE')
statsData1["broker"] = statsData1["account"].str[:5]
statsData1['count'] = statsData1.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

statsData2 = pd.DataFrame(statsData2)
statsData2['exchange'] = np.where(statsData2['secid'] >= 2000000, 'SZE', 'SSE')
statsData2["broker"] = statsData2["account"].str[:5]
statsData2['count'] = statsData2.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

statsData3 = pd.DataFrame(statsData3)
statsData3['exchange'] = np.where(statsData3['secid'] >= 2000000, 'SZE', 'SSE')
statsData3["broker"] = statsData3["account"].str[:5]
statsData3['count'] = statsData3.groupby(['broker', 'exchange', 'market orders'])['secid'].transform('count')

print(datetime.datetime.now() - startTm)

20201228
20201229
20201230
20201231
20210104
20210105
20210106
20210107
20210108
0:48:31.369935


In [6]:
summaryData = statsData3.copy()
summaryData['count'] = summaryData.groupby(['broker', 'exchange'])['secid'].transform('count')
summaryData['# of other orders'] = np.where(summaryData['market orders'] == 0, '0',
                                   np.where(summaryData['market orders'] <= 2, '1-2',
                                   np.where(summaryData['market orders'] <= 5, '3-5', '>= 6')))
summaryData['% count'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['secid'].transform('count')
summaryData['% count'] = summaryData['% count'] / summaryData['count']
summaryData['notional of other orders'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['market ntl'].transform('mean')
summaryData['# of total orders'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['market orders'].transform('sum')
summaryData['% orders in front'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['front orders'].transform('sum')
summaryData['% orders in front'] = summaryData['% orders in front'] / summaryData['# of total orders']

summaryData['notional of other orders clip'] = summaryData['market ntl'].clip(0, 5000000)
summaryData['notional of other orders clip'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['notional of other orders clip'].transform('mean')
summaryData['% notional in front'] = summaryData['front ntl'].clip(0, 5000000)
summaryData['% notional in front'] = summaryData.groupby(['broker', 'exchange', '# of other orders'])['% notional in front'].transform('mean')
summaryData['% notional in front'] = summaryData['% notional in front'] / summaryData['notional of other orders clip']

savePath = r'L:\orderLog\result\marketSpeed'
a = summaryData.groupby(['broker', 'exchange', 'count', '# of other orders'])[['% count', 'notional of other orders', '% orders in front', '% notional in front']].mean().reset_index()

summaryData = summaryData.groupby(['broker', 'exchange', 'count', '# of other orders'])[['% count', 'notional of other orders', '% orders in front', '% notional in front']].mean()
for col in [ 'notional of other orders']:
    summaryData[col] = summaryData[col].astype('int64')
for col in ['% count', '% orders in front', '% notional in front']:
    summaryData[col] = summaryData[col].fillna(0)
    summaryData[col] = summaryData[col].apply(lambda x: '%.0f%%'%(x*100))
from IPython.display import display, HTML
HTML(summaryData.to_html())

In [36]:
checkData[(checkData['sta'] == 'else') & (checkData['firstBroker'] != ' ')]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group,statusLs,TradePriceLs,TradeQtyLs,ars,mrstaat,mrstauc,isMsg,TradeNtl,exchange,brokerNum,brokerLs,order,groupPrice,OrderNtl,firstclock,firstBroker,sta,tradeNtl
241922,NaN,NaN,NaN,NaN,1.0,6200.0,98000.0,NaN,NaN,NaN,NaN,NaN,2000961,NaN,20201201,NaN,34243047.0,42948.0,"(0,)","(0,)","(0,)",1.0,NaN,NaN,1.0,0.0,SZE,1.0,[zs_8967],79355.0,98000.0,60760.0,1.606806e+15,zs_8967,else,0.0
384795,NaN,NaN,NaN,NaN,2.0,500.0,458800.0,NaN,NaN,NaN,NaN,NaN,2300487,NaN,20201201,NaN,1787257.0,81274.0,"(0,)","(0,)","(0,)",3.0,NaN,NaN,1.0,0.0,SZE,1.0,[zs_5456],53052.0,458500.0,22940.0,1.606806e+15,zs_5456,else,0.0
716156,NaN,NaN,NaN,NaN,1.0,1000.0,114900.0,NaN,NaN,NaN,NaN,NaN,2002449,NaN,20201203,NaN,22781966.0,47453.0,"(0,)","(0,)","(0,)",1.0,NaN,NaN,1.0,0.0,SZE,1.0,[zs_5232],4502.0,114900.0,11490.0,1.606978e+15,zs_5232,else,0.0
1970097,NaN,NaN,NaN,NaN,1.0,900.0,96300.0,NaN,NaN,NaN,NaN,NaN,2000157,NaN,20201211,NaN,78925535.0,44431.0,"(0,)","(0,)","(0,)",1.0,NaN,NaN,1.0,0.0,SZE,1.0,[zs_9435],110882.0,96300.0,8667.0,1.607657e+15,zs_9435,else,0.0


In [37]:
checkData[(checkData['sta'] == 'else') & (checkData['firstBroker'] == ' ')]

,SecurityID,ApplSeqNum,clockAtArrival,sequenceNo,Side,OrderQty,Price,cum_volume,TransactTime,updateType,TradePrice,TradeQty,secid,StockSeqNum,date,startVolume,vai,group,statusLs,TradePriceLs,TradeQtyLs,ars,mrstaat,mrstauc,isMsg,TradeNtl,exchange,brokerNum,brokerLs,order,groupPrice,OrderNtl,firstclock,firstBroker,sta,tradeNtl
21,1.0,2397748.0,1.606700e+15,14074671.0,1.0,300.0,200000.0,27031371.0,93720540.0,0.0,0.0,0.0,2000001,1.023977e+08,20201130,27031371.0,27021371.0,44597.0,"(0, 3)","(0, 0)","(0, 0)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,206400.0,6000.0,1.606700e+15,,else,0.0
23,1.0,2399623.0,1.606700e+15,14086472.0,1.0,6900.0,206200.0,27137371.0,93721160.0,0.0,0.0,0.0,2000001,1.023996e+08,20201130,27137371.0,27129357.0,44598.0,"(0, 4, 3)","(0, 206200, 0)","(0, 400, 0)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,206000.0,142278.0,1.606700e+15,,else,8248.0
26,1.0,2399684.0,1.606700e+15,14086785.0,1.0,200.0,206600.0,27137371.0,93721180.0,0.0,0.0,0.0,2000001,1.023997e+08,20201130,27137371.0,27129357.0,44598.0,"(0, 4)","(0, 206600)","(0, 200)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,206000.0,4132.0,1.606700e+15,,else,4132.0
27,1.0,2399685.0,1.606700e+15,14086793.0,1.0,1400.0,206000.0,27137371.0,93721180.0,0.0,0.0,0.0,2000001,1.023997e+08,20201130,27137371.0,27129357.0,44598.0,"(0, 4)","(0, 206000)","(0, 1400)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,206000.0,28840.0,1.606700e+15,,else,28840.0
28,1.0,2399687.0,1.606700e+15,14086813.0,1.0,1500.0,206000.0,27137371.0,93721180.0,0.0,0.0,0.0,2000001,1.023997e+08,20201130,27137371.0,27129357.0,44598.0,"(0, 4)","(0, 206000)","(0, 1500)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,206000.0,30900.0,1.606700e+15,,else,30900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2281811,300815.0,15582069.0,1.607665e+15,95624105.0,1.0,200.0,820200.0,1433476.0,132821090.0,0.0,0.0,0.0,2300815,3.008152e+13,20201211,1433476.0,1433176.0,128450.0,"(0, 3)","(0, 0)","(0, 0)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,820200.0,16404.0,1.607665e+15,,else,0.0
2281821,300815.0,15711324.0,1.607665e+15,96453046.0,1.0,200.0,820700.0,1448076.0,133002100.0,0.0,0.0,0.0,2300815,3.008152e+13,20201211,1448076.0,1447876.0,128455.0,"(0, 4)","(0, 820500)","(0, 200)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,820600.0,16414.0,1.607665e+15,,else,16410.0
2281833,300815.0,16893078.0,1.607666e+15,104264864.0,1.0,200.0,820100.0,1520376.0,134630510.0,0.0,0.0,0.0,2300815,3.008152e+13,20201211,1520376.0,1520076.0,128460.0,"(0, 4)","(0, 820100)","(0, 200)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,820100.0,16402.0,1.607666e+15,,else,16402.0
2281836,300815.0,18628848.0,1.607667e+15,116017031.0,2.0,100.0,818000.0,1595390.0,141241980.0,0.0,0.0,0.0,2300815,3.008152e+13,20201211,1595390.0,1595090.0,128461.0,"(0, 4)","(0, 818000)","(0, 100)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,[ ],NaN,818000.0,8180.0,1.607667e+15,,else,8180.0
